In [1]:
from pocket_coffea.lib.categorization import *
from pocket_coffea.lib.cut_functions import *
from pocket_coffea.parameters.cuts.preselection_cuts import *
from pocket_coffea.lib.cut_definition import Cut

In [2]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak

filename = "root://xrootd-cms.infn.it///store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/87DEE912-70CF-A549-B10B-1A229B256E88.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=1000).events()
events

<NanoEventsArray [<event 1:40419:40418002>, ... ] type='1000 * event'>

In [3]:
jet_cut =  lambda events, params, **kwargs: events.Jet.pt > params["pt"]
two_jets =  lambda events, params, **kwargs: ak.sum(events.Jet.pt>50, axis=1) > 3

In [4]:
catstd = StandardSelection(
        {"passthrough" : [passthrough],
             "jet_pt": [
                 Cut("jet_pt_large", {"pt":50}, jet_cut, collection="Jet"), 
                 Cut("ev_2_jet", {}, two_jets)]
        })

In [5]:
catstd.prepare(events, "2018","ttHTobb", isMC=True)

In [6]:
catstd.storage

MaskStorage(dim=2, masks=['ev_2_jet__3788061389622597362', 'jet_pt_large__7405548787399155291', 'passthrough__-7169401299319181905'])

In [7]:
catstd.get_mask("jet_pt")[0:10].tolist()

[[True, True, True, True, True, False, False],
 [False, False, False, False, False],
 [False, False, False, False],
 [False, False, False, False, False, False, False, False],
 [False, False, False, False, False, False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False, False, False, False],
 [True, True, True, True, False, False, False],
 [False, False, False, False, False, False, False],
 [False, False, False, False, False, False, False, False]]

In [8]:
catstd.get_mask("jet_pt") & ak.sum(events.Electron.pt> 20)>=1

<Array [[False, False, ... False, False]] type='1000 * var * bool'>

In [31]:
catstd.storage.dim

2

# Check product of dim=2 cuts

In [11]:
jets_pt = events.Jet.pt> 40
jets_eta = events.Jet.eta > 2
njets = ak.sum(events.Jet.pt > 40, axis=1) >=2

In [17]:
masks = [jets_pt, jets_eta]

In [18]:
masks

[<Array [[True, True, True, ... False, False]] type='1000 * var * bool'>,
 <Array [[False, False, ... False, False]] type='1000 * var * bool'>]

In [20]:
ak.prod(ak.concatenate([ak.singletons(m) for m in masks], axis=-1), axis=-1) == 1

<Array [[False, False, ... False, False]] type='1000 * var * bool'>

# Multicut

In [9]:
cat = CartesianSelection(
        multicuts = [
            MultiCut(name="Njets",
                     cuts=[
                         get_nObj_eq(4, 15., "Jet"),
                         get_nObj_eq(5, 15., "Jet"),
                         get_nObj_min(6, 15., "Jet"),
                     ],
                     cuts_names=["4j","5j","6j"]),
            MultiCut(name="jet_pt",
                    cuts=[
                         Cut("jet_pt_large", {"pt":20}, jet_cut, collection="Jet"),
                         Cut("jet_pt_large", {"pt":30}, jet_cut, collection="Jet"),
                         Cut("jet_pt_large", {"pt":40}, jet_cut, collection="Jet"),
                         Cut("jet_pt_large", {"pt":50}, jet_cut, collection="Jet")
                     ],
                     cuts_names=["pt20", "pt30","pt40", "pt50"])
        ],
        common_cats = catstd
    )

In [11]:
cat.categories

['passthrough',
 'jet_pt',
 '4j_pt20',
 '4j_pt30',
 '4j_pt40',
 '4j_pt50',
 '5j_pt20',
 '5j_pt30',
 '5j_pt40',
 '5j_pt50',
 '6j_pt20',
 '6j_pt30',
 '6j_pt40',
 '6j_pt50']

In [12]:
cat.prepare(events, "2018","ttHTobb", isMC=True)

In [13]:
cat.common_cats.storage

MaskStorage(dim=2, masks=['ev_2_jet__3788061389622597362', 'jet_pt_large__7405548787399155291', 'passthrough__-7169401299319181905'])

In [14]:
for c, m in cat.get_masks():
    print(c)
    print(m)

passthrough
[[True, True, True, True, True, True, True, ... True, True, True, True, True, True]]
jet_pt
[[True, True, True, True, True, False, ... False, False, False, False, False, False]]
4j_pt20
[[False, False, False, False, False, False, ... False, False, False, False, False]]
4j_pt30
[[False, False, False, False, False, False, ... False, False, False, False, False]]
4j_pt40
[[False, False, False, False, False, False, ... False, False, False, False, False]]
4j_pt50
[[False, False, False, False, False, False, ... False, False, False, False, False]]
5j_pt20
[[False, False, False, False, False, False, ... False, False, False, False, False]]
5j_pt30
[[False, False, False, False, False, False, ... False, False, False, False, False]]
5j_pt40
[[False, False, False, False, False, False, ... False, False, False, False, False]]
5j_pt50
[[False, False, False, False, False, False, ... False, False, False, False, False]]
6j_pt20
[[True, True, True, True, True, True, True, ... True, True, True, 